In [ ]:
from scipy.stats.qmc import LatinHypercube as LHSampler
from matplotlib import pyplot as plt
import numpy as np
from src.utils import plotLHS, concat, H, F
from src.eLHS import *
import json

# setup sampler
LHS = LHSampler(d = 2)

In [ ]:
N3, M3 = 10, 5
ss3 = LHS.random(N3)
pss3 = PartialeLHS(ss3, M3)
plotLHS(pss3, False, True)
hbar3 = sudoku_identity(pss3)

### Karash-Kuhn-Tucker Solver for LSSP
E pensare che mi ha dato 20...

In [ ]:
from scipy.optimize import minimize

# Objective Function
def objective(Q):
    return Q[0]

def constraint_eq(Q):
    pass 

def build_constraint_ineq(S: np.ndarray, q, j, alternative = 1, H = H):
    if alternative == 1:
        def constraint_ineq(unknowns):
            Q = unknowns[0]
            N = S.shape[0]
            if q/(N+Q)> 1: 
                return 0
            return -np.sum(H(S[:, j] - q/(N+Q)) * H((q+1)/(N+Q) - S[:, j]) - 1)
        return constraint_ineq
    elif alternative == 2:
        def constraint_ineq(unknowns):
            Q = unknowns[0]
            N = S.shape[0]
            return H(1 - q/(N+Q)) * -(np.sum(H(S[:, j] - q/(N+Q)) * H((q+1)/(N+Q) - S[:, j])) - 1) 
        return constraint_ineq
    

constraint_ineqs = []
for q in range(2 * N3):
    for j in range(ss3.shape[1]):
        constraint_ineqs.append({'type': 'ineq', 'fun': build_constraint_ineq(ss3, q, j, 2)})

# constraint_ineqs.append(
#     {'type': 'ineq', 'fun': lambda Q: Q[0] - 1}
# )
# constraint_ineqs.append(
#     {'type': 'ineq', 'fun': lambda Q: N3 + Q[0]}
# )

initial_guess = [14]

result = minimize(
    objective,
    initial_guess,
    constraints = constraint_ineqs, 
    bounds = [(1, N3-1)], 
    method='SLSQP', 
    tol=1e-6,
    options={
        # 'maxiter': 10000
        'disp': True
    }

)

print(result)

In [ ]:
# SOLVER STEPPER (for Sample Set ss3)
# Qs = np.linspace(1, N3-1, N3-1)
P = 2 
guessQ = 5
unconsistencies = np.zeros((P))
print("    q j  | passed")
print("    ---  | ------")
for j in range(P):
    for q in range(N3):
        if(constraint_ineqs[q * P + j]['fun']([guessQ]) >= 0):
            print("=> ", q, j, " | ✅")
        else:
            print("=> ", q, j, " | ❌")
            unconsistencies[j] += 1

print("END")

print(unconsistencies)


In [ ]:
a, b = 0, N3
xs = np.linspace(a, b, N3+1)
for i in range(N3 * 2):
    ys = [constraint_ineqs[i]['fun']([x]) for x in xs]
    # plt.plot(xs, ys)
    plt.scatter(xs, ys, s=0.5)